## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [1]:
# Your code here - remember to use markdown cells for comments as well!
import pandas as pd
import urllib.request, urllib.parse, urllib.error
import requests
import json

In [2]:
API_key = '294f4c0e6d0861470f47e84017c8e2c1'

In [3]:
#The first API request is to obtain a list of movies by the released_date

search_year = 2018

#This request pulls the first page of the search request
resp = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' + str(API_key) +'&language=en-US&sort_by=popularity.desc&primary_release_year='+ str(search_year))
data_p1 = dict(resp.json())

#We can also use this request to see how many pages there are in the search, 
#we will need to add all the pages of the search to the dataframe
search_pages = data_p1['total_pages']
data = pd.DataFrame.from_dict(data_p1['results'])

In [4]:
## Takes ~30 seconds ##

#This step is slow, there is a lot of data being requested.
url ='https://api.themoviedb.org/3/discover/movie?api_key=' + str(API_key) +'&language=en-US&sort_by=popularity.desc&primary_release_year='+ str(search_year)+'&page='
for page in range(2,search_pages+1):
    resp = requests.get(url + str(page))
    temp = dict(resp.json())
    df_by_page = pd.DataFrame.from_dict(temp['results'])
    data = pd.concat([data,df_by_page], ignore_index=True)

In [5]:
# Because there are 10,000 movies in 2017.  (I think that's the max value because all years max out at 10,000 it seems)
# I filtered by popularity so we can use the top <integer value> we want to select

data = data.sort_values(by='popularity', ascending=False)
data = data.set_index('id')
data = data.drop(columns = 'genre_ids')
data = data.drop_duplicates()

In [127]:
# Create a list of all the movies from the search above. Which is in decending popularity order.
movies_index = data.index.values.tolist()
len(movies_index)

10000

In [7]:
#Here is an example of the API request data.  
#If we want to change our dataframes, to target more data, it would be easy to add in the function in an upcoming step
resp = requests.get('https://api.themoviedb.org/3/movie/' + str(movies_index[0]) + '?api_key=' + str(API_key))
dict(resp.json())

{'adult': False,
 'backdrop_path': '/vc8bCGjdVp0UbMNLzHnHSLRbBWQ.jpg',
 'belongs_to_collection': {'id': 531241,
  'name': 'Spider-Man (Avengers) Collection',
  'poster_path': '/nogV4th2P5QWYvQIMiWHj4CFLU9.jpg',
  'backdrop_path': '/zh72eVJTD5EssStCsNHqELuh6uD.jpg'},
 'budget': 175000000,
 'genres': [{'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'},
  {'id': 878, 'name': 'Science Fiction'},
  {'id': 18, 'name': 'Drama'}],
 'homepage': 'http://www.spidermanhomecoming.com',
 'id': 315635,
 'imdb_id': 'tt2250912',
 'original_language': 'en',
 'original_title': 'Spider-Man: Homecoming',
 'overview': 'Following the events of Captain America: Civil War, Peter Parker, with the help of his mentor Tony Stark, tries to balance his life as an ordinary high school student in Queens, New York City, with fighting crime as his superhero alter ego Spider-Man as a new threat, the Vulture, emerges.',
 'popularity': 55.385,
 'poster_path': '/c24sv2weTHPsmDa7jEMN0m2P3RT.jpg',
 'production_co

In [94]:
#Defines a function to then arrange the data into a dictionary for a given movie.  

def API_request(movie):
    resp = requests.get('https://api.themoviedb.org/3/movie/' + str(movie) + '?api_key=' + str(API_key))
    if resp.status_code == requests.codes.ok:
        temp = dict(resp.json())
        movie = {}
        movie['imdb_id'] = temp['imdb_id']
        movie['release_date'] = pd.to_datetime(temp['release_date'])
        movie['genres'] = [genre['name'] for genre in temp['genres']]
        movie['title'] = temp['title']
        movie['budget'] = int(temp['budget'])
        movie['revenue'] = int(temp['revenue'])
        movie['ROI'] =  int(temp['revenue']) - int(temp['budget'])
        movie['runtime_mins'] = temp['runtime']
        return movie

In [9]:
#See an example of what the API_request provides per movie
print(API_request(movies_index[0]))

{'imdb_id': 'tt2250912', 'release_date': Timestamp('2017-07-05 00:00:00'), 'genres': ['Action', 'Adventure', 'Science Fiction', 'Drama'], 'title': 'Spider-Man: Homecoming', 'budget': 175000000, 'revenue': 880166924, 'ROI': 705166924, 'runtime_mins': 133}


In [153]:
## Execution time varies on how large our subset is ##

#Define how many of the top most popular movies we would like to use in our dataset:
subset_lenght = 100

#Shrinks the movies_index which is arranged from most popular to least popular, 
#and calls the first "subset_lenght" amount.
imdb_ID_subset = movies_index[:subset_lenght]

#Make requests to the API for the movie data
imbd_API_data = list(map(lambda x: API_request(x),imdb_ID_subset))

In [154]:
#Create a dataframe of the data obtained
df = pd.DataFrame.from_dict(imbd_API_data)
df.head()

,imdb_id,release_date,genres,title,budget,revenue,ROI,runtime_mins
0,tt2250912,2017-07-05,"[Action, Adventure, Science Fiction, Drama]",Spider-Man: Homecoming,175000000,880166924,705166924,133
1,tt0974015,2017-11-15,"[Action, Adventure, Fantasy, Science Fiction]",Justice League,300000000,657924295,357924295,120
2,tt1856101,2017-10-04,"[Science Fiction, Drama]",Blade Runner 2049,150000000,259239658,109239658,163
3,tt3501632,2017-10-25,"[Action, Adventure, Comedy, Fantasy]",Thor: Ragnarok,180000000,853977126,673977126,131
4,tt1485796,2017-12-20,[Drama],The Greatest Showman,84000000,432844677,348844677,105


In [155]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   imdb_id       100 non-null    object        
 1   release_date  100 non-null    datetime64[ns]
 2   genres        100 non-null    object        
 3   title         100 non-null    object        
 4   budget        100 non-null    int64         
 5   revenue       100 non-null    int64         
 6   ROI           100 non-null    int64         
 7   runtime_mins  100 non-null    int64         
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 6.4+ KB


In [156]:
#Split the genres column into individual rows and set the index to "imbd_id"
s = df.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
cleaned_df = df.drop('genres',axis=1).join(s).set_index('imdb_id')
cleaned_df.head()

,release_date,title,budget,revenue,ROI,runtime_mins,genre
imdb_id,,,,,,,
tt2250912,2017-07-05,Spider-Man: Homecoming,175000000,880166924,705166924,133,Action
tt2250912,2017-07-05,Spider-Man: Homecoming,175000000,880166924,705166924,133,Adventure
tt2250912,2017-07-05,Spider-Man: Homecoming,175000000,880166924,705166924,133,Science Fiction
tt2250912,2017-07-05,Spider-Man: Homecoming,175000000,880166924,705166924,133,Drama
tt0974015,2017-11-15,Justice League,300000000,657924295,357924295,120,Action


In [157]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 295 entries, tt2250912 to tt3829920
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   release_date  295 non-null    datetime64[ns]
 1   title         295 non-null    object        
 2   budget        295 non-null    int64         
 3   revenue       295 non-null    int64         
 4   ROI           295 non-null    int64         
 5   runtime_mins  295 non-null    int64         
 6   genre         295 non-null    object        
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 18.4+ KB
